# Validación Numérica del Modo QNM (ℓ=2, m=2)
## Corrección por la Teoría del Espacio Dinámico Rotacional (EDR)

Este notebook calcula:
1. El modo QNM GR base usando aproximación WKB (3er orden).
2. El potencial perturbado por EDR.
3. El corrimiento $δ\omega$.
4. La razón $α_{flow} = δ\omega / \omega_0$.


In [ ]:
# IMPORTS
import numpy as np
from scipy.integrate import simps
from scipy.optimize import minimize_scalar
import matplotlib.pyplot as plt

## 1. Potencial de Regge–Wheeler para ℓ = 2


In [ ]:
def V_RW(r, l=2, M=1.0):
    return (1 - 2*M/r) * (l*(l+1)/r**2 - 6*M/r**3)


## 2. Localizar el máximo del potencial


In [ ]:
def find_rmax():
    f = lambda r: -V_RW(r)
    sol = minimize_scalar(f, bounds=(2.2, 10), method='bounded')
    return sol.x

r0 = find_rmax()
V0 = V_RW(r0)
r0, V0

## 3. Cálculo del modo GR base usando WKB 3er orden


In [ ]:
def omega_GR():
    n = 0
    deriv = 1e-4
    V2 = (V_RW(r0+deriv) - 2*V_RW(r0) + V_RW(r0-deriv)) / deriv**2
    w = np.sqrt(V0 - 1j*(n+1/2)*np.sqrt(-2*V2))
    return w

omega0 = omega_GR()
omega0

## 4. Definir la perturbación EDR: \( \delta V = \eta \, \Omega^2 \, e^{-r/10} \)


In [ ]:
eta = 0.02
Omega = 0.1

def deltaV(r):
    return eta * Omega**2 * np.exp(-r/10)


## 5. Construir la base normalizada del modo GR


In [ ]:
# Eje r*
r = np.linspace(2.1, 80, 4000)

# Aproximación de la solución local
Psi = np.exp(-((r - r0)**2) / 3)

# Normalizar
Psi /= np.sqrt(simps(Psi**2, r))

plt.plot(r, Psi)
plt.title("Modo base normalizado Psi_0(r)")
plt.xlabel("r")
plt.ylabel("Psi")
plt.show()

## 6. Evaluar la integral: \( \delta\omega = \frac{1}{2\omega_0} \int dr\, \Psi_0 \, \delta V \, \Psi_0 \)


In [ ]:
integral = simps(Psi * deltaV(r) * Psi, r)
delta_omega = integral / (2 * omega0)
alpha_flow = delta_omega / omega0

delta_omega, alpha_flow

## 7. Resultado final

Mostramos:
- Frecuencia GR base \(\omega_0\)
- Corrimiento \(\delta \omega\)
- Parámetro de flujo EDR: \(\alpha_{flow}\)


In [ ]:
print("ω0 (GR)      =", omega0)
print("δω (EDR)     =", delta_omega)
print("α_flow       =", alpha_flow)